In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from war_functions.pecota_tables import *

from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *

from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *

from odds_and_other_projections import *

team_map = {
    'Giants' : 'San Francisco Giants',
    'Astros' : 'Houston Astros',
    'Brewers': 'Milwaukee Brewers',
    'Dodgers': 'Los Angeles Dodgers',
    'Rays' : 'Tampa Bay Rays',
    'Red Sox': 'Boston Red Sox',
    'White Sox': 'Chicago White Sox',
    'Padres': 'San Diego Padres',
    'Athletics': 'Oakland Athletics',
    'Yankees' : 'New York Yankees',
    'Mariners': 'Seattle Mariners',
    'Reds': 'Cincinnati Reds',
    'Blue Jays' : 'Toronto Blue Jays',
    'Mets' : 'New York Mets',
    'Phillies': 'Philadelphia Phillies',
    'Angels': 'Los Angeles Angels',
    'Braves': 'Atlanta Braves',
    'Cardinals': 'St. Louis Cardinals',
    'Tigers': 'Detroit Tigers',
    'Cubs': 'Chicago Cubs',
    'Indians': 'Cleveland Indians',
    'Guardians': 'Cleveland Guardians',
    'Nationals': 'Washington Nationals',
    'Rockies': 'Colorado Rockies',
    'Marlins': 'Miami Marlins',
    'Royals': 'Kansas City Royals',
    'Twins': 'Minnesota Twins',
    'Pirates': 'Pittsburgh Pirates',
    'Rangers': 'Texas Rangers',
    'Orioles': 'Baltimore Orioles',
    'Diamondbacks': 'Arizona Diamondbacks',
    'D-backs' : 'Arizona Diamondbacks'
}

In [2]:
# Retrieving necessary data

active_rosters = retrieve_all_active_rosters(file_name = None)
todays_games = retrieve_todays_games_info()
current_year_war = retrieve_current_year_WAR()
pt = load_combined_pecota_table()
odds = retrieve_odds()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST driver version for 100.0.4896
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [11]:
tables = pd.read_html('https://www.espn.com/mlb/stats/team')
runs_scored_table = pd.merge(tables[0], tables[1], left_on = tables[0].index, right_on = tables[1].index)
runs_scored_table.drop('key_0', axis = 1, inplace = True)
runs_scored_table = runs_scored_table[['Team', 'GP', 'R']]
runs_scored_table['Runs_162'] = runs_scored_table.R * (162.0/runs_scored_table.GP)

tables = pd.read_html(f'https://www.foxsports.com/mlb/team-stats?category=pitching&season={dt.date.today().year}&seasonType=reg')
runs_allowed_table = tables[1]
runs_allowed_table = runs_allowed_table.iloc[:,[1,2,13]]
runs_allowed_table.columns = ['Team', 'Games', 'Runs_Allowed']
runs_allowed_table['Runs_Allowed_162'] = runs_allowed_table.Runs_Allowed * (162.0/runs_allowed_table.Games)
runs_allowed_table['Team'] = runs_allowed_table.Team.apply(lambda x: team_map[x])

merged = pd.merge(runs_scored_table, runs_allowed_table, on = 'Team')
merged = merged[['Team', 'Games', 'Runs_162', 'Runs_Allowed_162']]

current_year_hitting = retrieve_historical_hitting_tables(2022, file_name = None)
hitting_reg = load_linear_regression("data/hitting_regression.pickle")
cluster_luck_hitting = calculate_predicted_cluster_luck_run_adjustment_hitting(hitting_reg, current_year_hitting)
cluster_luck_hitting = cluster_luck_hitting[['Team', 'GP', 'run_adjust']]
cluster_luck_hitting.columns = ['Team', 'Games', 'Offensive_Adjustment']
cluster_luck_hitting['Offensive_Adjustment'] = cluster_luck_hitting.Offensive_Adjustment * (162.0/cluster_luck_hitting.Games)

current_year_pitching = retrieve_historical_pitching_tables(dt.date.today().year, file_name = None)
pitching_reg = load_linear_regression('./data/pitching_regression.pickle')
cluster_luck_pitching = calculate_predicted_cluster_luck_run_adjustment_pitching(pitching_reg, current_year_pitching)

/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/hsinger24/Desktop/Programming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Team,Games,Runs_Allowed,Runs_Allowed_162
0,New York Mets,15,42,453.600000
1,Los Angeles Dodgers,13,33,411.230769
2,Toronto Blue Jays,14,53,613.285714
3,San Francisco Giants,14,37,428.142857
4,San Diego Padres,15,49,529.200000
5,Los Angeles Angels,14,61,705.857143
6,Oakland Athletics,15,58,626.400000
7,Milwaukee Brewers,14,53,613.285714
8,St. Louis Cardinals,12,37,499.500000
9,Seattle Mariners,14,48,555.428571


In [12]:
starting_rotations, failed_to_find_pitchers = retrieve_starting_rotations_WAR(pt, current_year_war)

/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/starting_rotations_WAR.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_pitchers['WAR_proj'] = 0
/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/starting_rotations_WAR.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starting_pitchers['WAR'] = 0
/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/starting_rotations_WAR.py:45: FutureWarning: elementwise comparison failed; returning scalar instead, but in the futu

In [49]:
sp_adjustments = calculate_sp_adjustment(todays_games, starting_rotations, pt, frac_season = 0.08)

All names failed for       Unnamed: 0.1    bpid      name name_wo_a name_alt_1  name_alt_2  \
3208          3208  108959  Sam Long  Sam Long        NaN         NaN   

      name_alt_3  name_alt_4  
3208         NaN         NaN   Pecota WAR is: 0.3


/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/adjusted_war_today.py:50: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sp_home in names[col].values:
/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/adjusted_war_today.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sp_away in names[col].values:


In [45]:
overall_war_predictions_preseason = pd.read_csv('data/overall_war_predictions_preseason.csv')
active_roster_war, failed_to_find_players = calculate_active_roster_war_table(active_rosters, overall_war_predictions_preseason, current_year_war, pt, 2022, 0.08)

/Users/hsinger24/Desktop/Baseball-Betting/daily_adjustments/adjusted_war_today.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if player_name in names[col].values:
